<a href="https://colab.research.google.com/github/AnitLearner/PLANT-DISEASE-CLASSSIFIER-USING-ALEXNET/blob/main/Traffic_Signal_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
import os
os.environ['KAGGEL_CONFIG_DIR'] = '/content'
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
import zipfile
zip_ref = zipfile.ZipFile('/content/gtsrb-german-traffic-sign.zip', 'r')
zip_ref.extractall('/content/gtsrb-german-traffic-sign')
zip_ref.close()

 96% 589M/612M [00:08<00:00, 29.0MB/s]
100% 612M/612M [00:08<00:00, 74.1MB/s]


In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
np.random.seed(42)

from matplotlib import style
style.use('fivethirtyeight')

In [3]:
data_dir = "/content/gtsrb-german-traffic-sign"
train_path = '/content/gtsrb-german-traffic-sign/Train'
test_path = '/content/gtsrb-german-traffic-sign/Test'

# Resizing the images to 30x30x3
IMG_HEIGHT = 30
IMG_WIDTH = 30
channels = 3
NUM_CATEGORIES = len(os.listdir(train_path))
NUM_CATEGORIES

43

In [4]:
# Label Overview
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)',
            2:'Speed limit (50km/h)',
            3:'Speed limit (60km/h)',
            4:'Speed limit (70km/h)',
            5:'Speed limit (80km/h)',
            6:'End of speed limit (80km/h)',
            7:'Speed limit (100km/h)',
            8:'Speed limit (120km/h)',
            9:'No passing',
            10:'No passing veh over 3.5 tons',
            11:'Right-of-way at intersection',
            12:'Priority road',
            13:'Yield',
            14:'Stop',
            15:'No vehicles',
            16:'Veh > 3.5 tons prohibited',
            17:'No entry',
            18:'General caution',
            19:'Dangerous curve left',
            20:'Dangerous curve right',
            21:'Double curve',
            22:'Bumpy road',
            23:'Slippery road',
            24:'Road narrows on the right',
            25:'Road work',
            26:'Traffic signals',
            27:'Pedestrians',
            28:'Children crossing',
            29:'Bicycles crossing',
            30:'Beware of ice/snow',
            31:'Wild animals crossing',
            32:'End speed + passing limits',
            33:'Turn right ahead',
            34:'Turn left ahead',
            35:'Ahead only',
            36:'Go straight or right',
            37:'Go straight or left',
            38:'Keep right',
            39:'Keep left',
            40:'Roundabout mandatory',
            41:'End of no passing',
            42:'End no passing veh > 3.5 tons' }

In [5]:
image_data = []
image_labels = []

for i in range(NUM_CATEGORIES):
    path = data_dir + '/Train/' + str(i)
    images = os.listdir(path)

    for img in images:
        try:
            image = cv2.imread(path + '/' + img)
            image_fromarray = Image.fromarray(image, 'RGB')
            resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
            image_data.append(np.array(resize_image))
            image_labels.append(i)
        except:
            print("Error in " + img)

# Changing the list to numpy array
image_data = np.array(image_data)
image_labels = np.array(image_labels)

print(image_data.shape, image_labels.shape)

(39209, 30, 30, 3) (39209,)


In [6]:
## Shuffling the training data
shuffle_indexes = np.arange(image_data.shape[0])
np.random.shuffle(shuffle_indexes)
image_data = image_data[shuffle_indexes]
image_labels = image_labels[shuffle_indexes]
## Splitting the data into train and validation set
X_train, X_val, y_train, y_val = train_test_split(image_data, image_labels, test_size=0.3, random_state=42, shuffle=True)

X_train = X_train/255
X_val = X_val/255

print("X_train.shape", X_train.shape)
print("X_valid.shape", X_val.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_val.shape)
## One hot encoding the labels
y_train = keras.utils.to_categorical(y_train, NUM_CATEGORIES)
y_val = keras.utils.to_categorical(y_val, NUM_CATEGORIES)

print("y_train.shape",y_train.shape)

X_train.shape (27446, 30, 30, 3)
X_valid.shape (11763, 30, 30, 3)
y_train.shape (27446,)
y_valid.shape (11763,)
y_train.shape (27446, 43)


In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from tensorflow.keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from tensorflow.keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU

In [8]:
DWS_model_TSR = keras.models.Sequential([
                  keras.layers.Conv2D(6, kernel_size=3, strides=1,  activation='relu', input_shape=(IMG_HEIGHT,IMG_WIDTH,channels), padding='same'), #Conv_Layer_1
                  #keras.layers.AveragePooling2D(), #S2
                  keras.layers.SeparableConv2D(2,kernel_size=3,strides=(1, 1), padding='same',dilation_rate=(1, 1),depth_multiplier=1,activation=None,use_bias=True),#DWSConv_Layer_2
                  keras.layers.Conv2D(2, kernel_size=3, strides=1,  activation='relu', input_shape=(IMG_HEIGHT,IMG_WIDTH,channels), padding='same'), # #Conv_Layer_3
                  #keras.layers.Conv2D(120, kernel_size=5, strides=1, activation='relu', padding='same'), #C5
                  keras.layers.SeparableConv2D(2,kernel_size=3,strides=(1, 1), padding='same',dilation_rate=(1, 1),depth_multiplier=1,activation=None,use_bias=True), #Conv_Layer_4
                  keras.layers.MaxPooling2D(), #S2
                  keras.layers.MaxPooling2D(), #S2
                  keras.layers.Flatten(), #Flatten
                  #keras.layers.Dense(20, activation='tanh'), #F6
                  keras.layers.Dense(43, activation='softmax') #Output layer
                  ])

In [9]:
# Compile the model
DWS_model_TSR.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
epochs=3
# Train the model
#history = model.fit(train_x, train_y, epochs=10, validation_data=(val_x, val_y))
# prompt: stop training if the there is no significant improvements in accuracy

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
history = DWS_model_TSR.fit(X_train, y_train,  epochs=epochs, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluate the model
from sklearn.metrics import accuracy_score
y_pred = DWS_model_TSR.predict(X_val)
y_pred = y_pred.argmax(axis=-1)
y_valn=y_val.argmax(axis=-1)
accuracy = accuracy_score(y_valn,y_pred)
print('Test Data accuracy: ',accuracy_score(y_valn,y_pred)*100)

Epoch 1/3
858/858 [==============================] - 59s 66ms/step - loss: 2.5542 - accuracy: 0.3131 - val_loss: 1.3281 - val_accuracy: 0.6225
Epoch 2/3
858/858 [==============================] - 48s 56ms/step - loss: 1.0239 - accuracy: 0.6952 - val_loss: 0.8237 - val_accuracy: 0.7528
Epoch 3/3
368/368 [==============================] - 9s 24ms/step
Test Data accuracy:  82.43645328572643
